In [1]:
!pip3 install -q datasets peft accelerate evaluate scikit-learn rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 w

In [2]:
!pip3 install -q --upgrade transformers

In [3]:
import torch
import torch.nn as nn
from datasets import load_dataset
import evaluate
from transformers import (
    GPT2Config,
    GPT2TokenizerFast,
    GPT2PreTrainedModel,
    GPT2Model,
    Trainer,
    TrainingArguments,
)
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

MAX_LENGTH     = 384
STRIDE         = 128
BATCH_SIZE     = 4
LEARNING_RATE  = 3e-5
NUM_EPOCHS     = 3
OUTPUT_DIR     = "gpt2-qa"

raw = load_dataset("squad", name="plain_text")
train_raw = raw["train"].select(range(87500))
val_raw   = raw["validation"].select(range(10000))
metric    = evaluate.load("squad")
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
tokenizer.add_special_tokens({"pad_token": "[PAD]"})
PAD_ID = tokenizer.pad_token_id

2025-05-20 04:47:39.254098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747716459.453751      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747716459.510371      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [4]:
class GPT2ForQuestionAnswering(GPT2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPT2Model(config)
        self.qa_outputs  = nn.Linear(config.hidden_size, 2)
        self.init_weights()

    def forward(self, input_ids, attention_mask=None, start_positions=None, end_positions=None):
        hidden_states = self.transformer(input_ids, attention_mask=attention_mask).last_hidden_state
        logits        = self.qa_outputs(hidden_states)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits   = end_logits.squeeze(-1)

        loss = None
        if start_positions is not None and end_positions is not None:
            loss_fct    = nn.CrossEntropyLoss()
            start_loss = loss_fct(start_logits, start_positions)
            end_loss   = loss_fct(end_logits,   end_positions)
            loss       = (start_loss + end_loss) / 2

        return {"loss": loss, "start_logits": start_logits, "end_logits": end_logits}

In [5]:
def prepare_features(examples):
    all_outputs = {
        "input_ids": [], "attention_mask": [],
        "start_positions": [], "end_positions": [],
        "id": [], "answers": []
    }

    for q, c, ex_id, ans in zip(
        examples["question"],
        examples["context"],
        examples["id"],
        examples["answers"],
    ):
        # single‐window tokenization, always truncating if needed
        enc = tokenizer(
            q,
            c,
            truncation=True,          # truncate either pair as needed
            max_length=MAX_LENGTH,
            padding="max_length",
            return_offsets_mapping=True,
        )

        input_ids      = enc["input_ids"]
        attention_mask = enc["attention_mask"]
        offsets        = enc["offset_mapping"]

        # pseudo‐CLS at PAD or fallback to 0
        cls_index = input_ids.index(PAD_ID) if PAD_ID in input_ids else 0

        # default span = CLS
        start_pos = cls_index
        end_pos   = cls_index

        # compute true span in this truncated window
        ans_start = ans["answer_start"][0]
        ans_text  = ans["text"][0]
        ans_end   = ans_start + len(ans_text)

        for idx, (o_start, o_end) in enumerate(offsets):
            if o_start <= ans_start < o_end:
                start_pos = idx
            if o_start < ans_end <= o_end:
                end_pos = idx
                break

        all_outputs["input_ids"].append(input_ids)
        all_outputs["attention_mask"].append(attention_mask)
        all_outputs["start_positions"].append(start_pos)
        all_outputs["end_positions"].append(end_pos)
        all_outputs["id"].append(ex_id)
        all_outputs["answers"].append(ans)

    return all_outputs


In [6]:
tokenized_train = train_raw.map(
    prepare_features,
    batched=True,
    remove_columns=train_raw.column_names,
)
tokenized_val = val_raw.map(
    prepare_features,
    batched=True,
    remove_columns=val_raw.column_names,
)

config = GPT2Config.from_pretrained("gpt2")
config.pad_token_id = PAD_ID
model = GPT2ForQuestionAnswering.from_pretrained("gpt2", config=config)
model.resize_token_embeddings(len(tokenizer))

device = (
    "mps" if torch.backends.mps.is_available() else
    "cuda" if torch.cuda.is_available() else "cpu"
)
print(f"Using device: {device}")
model.to(device)

Map:   0%|          | 0/87500 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at gpt2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Using device: cuda


GPT2ForQuestionAnswering(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (qa_outputs): Linear(in_features=768, out_features=2, bias=True)
)

In [7]:
def compute_qa_metrics(p):
    start_logits, end_logits = p.predictions
    preds, refs = [], []

    for i in range(len(start_logits)):
        start = int(start_logits[i].argmax())
        end   = int(end_logits[i].argmax())
        token_ids = tokenized_val[i]["input_ids"][start:end+1]
        text = tokenizer.decode(token_ids, skip_special_tokens=True)

        preds.append({
            "id": tokenized_val[i]["id"],
            "prediction_text": text
        })
        refs.append({
            "id": tokenized_val[i]["id"],
            "answers": tokenized_val[i]["answers"]
        })

    return metric.compute(predictions=preds, references=refs)


In [8]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=3e-5,
    warmup_steps=500,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,
    fp16=True,
    num_train_epochs=3,
    eval_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    save_strategy="epoch",
    save_total_limit=2,
    logging_strategy="steps",
    logging_steps=100,
    report_to=[]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_qa_metrics,
)

trainer.train()
results = trainer.evaluate()
print("Final evaluation:", results)

Epoch,Training Loss,Validation Loss,Exact Match,F1
1,1.846700,1.700001,55.160000,65.326009
2,1.393300,1.573778,58.750000,68.044909


Final evaluation: {'eval_loss': 1.5737777948379517, 'eval_exact_match': 58.75, 'eval_f1': 68.04490867385584, 'eval_runtime': 163.3849, 'eval_samples_per_second': 61.205, 'eval_steps_per_second': 7.651, 'epoch': 2.9991314285714283}


In [9]:
def eval_collate_fn(features):
    tensor_features = [
        {k: f[k] for k in ("input_ids", "attention_mask")}
        for f in features
    ]
    batch_inputs = trainer.data_collator(tensor_features)
    batch_meta = [{"id": f["id"], "answers": f["answers"], "input_ids": f["input_ids"]} for f in features]
    return batch_inputs, batch_meta

val_loader = DataLoader(
    tokenized_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=eval_collate_fn
)

model.eval()
all_preds, all_refs = [], []

with torch.no_grad():
    for batch_inputs, batch_meta in tqdm(val_loader, desc="Evaluating"):
        batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}
        outputs = model(**batch_inputs)

        start_logits = outputs["start_logits"].cpu().numpy()
        end_logits   = outputs["end_logits"].cpu().numpy()
        for i, meta in enumerate(batch_meta):
            s = int(start_logits[i].argmax())
            e = int(end_logits[i].argmax())
            span_ids = meta["input_ids"][s : e + 1]
            pred_text = tokenizer.decode(span_ids, skip_special_tokens=True)

            all_preds.append({
                "id": meta["id"],
                "prediction_text": pred_text
            })
            all_refs.append({
                "id": meta["id"],
                "answers": meta["answers"]
            })

results = metric.compute(predictions=all_preds, references=all_refs)
print(f"Exact Match: {results['exact_match']:.2f}, F1: {results['f1']:.2f}")

Evaluating:   0%|          | 0/2500 [00:00<?, ?it/s]

Exact Match: 58.74, F1: 68.03
